# **Brady Analysis**

The point of this exercise to to analyze past nfl data and see if brady elevated his players or not. this is done by first creating a baseline of the player with brady, and how he did without.

In [244]:
#import libraries
import numpy as np
import pandas as pd
import re

In [245]:
#import data undo dataframes for each respective year
#data is from NFLsavant.com
df13 = pd.read_csv('pbp-2013.csv')
df14 = pd.read_csv('pbp-2014.csv')
df15 = pd.read_csv('pbp-2015.csv')
df16 = pd.read_csv('pbp-2016.csv')
df17 = pd.read_csv('pbp-2017.csv')
df18 = pd.read_csv('pbp-2018.csv')
df19 = pd.read_csv('pbp-2019.csv')
df20 = pd.read_csv('pbp-2020.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [246]:
#create a function to find all players associated with the QB per year
def name_bin(df, name):
    df_bin = []
    for x in df[(df['PlayType'] == 'PASS') & (df['IsIncomplete'] == 0)]['Description']:
        if name in x:
            x_re = re.compile(r'(\w\.\w+)')
            player = x_re.findall(x)
            #print(player)
            if player[1] not in df_bin:
                df_bin.append(player[1])
    return df_bin

#create bins of players for each respective year
bin_2013 = name_bin(df13, 'T.BRADY')
bin_2014 = name_bin(df14, 'T.BRADY')
bin_2015 = name_bin(df15, 'T.BRADY')
bin_2016 = name_bin(df16, 'T.BRADY')
bin_2017 = name_bin(df17, 'T.BRADY')
bin_2018 = name_bin(df18, 'T.BRADY')
bin_2019 = name_bin(df19, 'T.BRADY')
bin_2020 = name_bin(df20, 'T.BRADY')


In [247]:
#create a function to total yards and touchdown for the given year
def get_stats(name, df, year):
    year = str(year)
    year = year[2:]
    #print(year)
    total_yards = 0
    total_tds = 0
    for index, x in df[(df['PlayType'] == 'PASS')  & \
                       (df['GameDate'].str.contains(str(year))) & (df['IsInterception'] == 0) \
                      & (df['IsIncomplete'] == 0) & (df['IsPenaltyAccepted'] == 0)].iterrows():
                           
        if name in x['Description']:
            x_re = re.compile(r'(\w\.\w+)')
            player = x_re.findall(x['Description'])
            if len(player) > 1:
                if (name == player[1]):
                    total_yards += x['Yards']
                    total_tds += x['IsTouchdown']
    return (year, total_yards, total_tds)

#tested for d.amendola  
da_yr, da_yds, da_tds = get_stats('D.AMENDOLA', df13, 2013)
print(da_yr, da_yds, da_tds)
            
            

13 633 2


In [248]:
da_test = df13[(df13['PlayType'] == 'PASS') & (df13['Description'].str.contains('D.AMENDOLA'))]
da_test.to_csv('datest.csv')

In [249]:
#created empty bins for comparison later
pre_brady = {}
with_brady = {}
post_brady = {}

#created lists for easier loop access
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
bins = [bin_2013, bin_2014, bin_2015, bin_2016, bin_2017, bin_2018, bin_2019, bin_2020]
dfs = [df13, df14, df15, df16, df17, df18, df19, df20]




In [250]:
#create the initial bin
for x in bin_2013:
    if x not in with_brady.keys():
        with_brady[x] = [get_stats(x, df13, 2013)]
#print(list(with_brady.items()))
#print(list(with_brady.keys()))

for s in range(1,3):
    for name in bins[s]:
        if name not in bins[s-1]:
            if name not in pre_brady.keys():
                pre_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                pre_brady[name] += [get_stats(name, dfs[s], years[s])]
        elif name not in bins[s+1]:
            if name not in post_brady.keys():
                post_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                post_brady[name] += [get_stats(name, dfs[s], years[s])]
        else:
            if name not in with_brady.keys():
                with_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                with_brady[name] += [get_stats(name, dfs[s], years[s])]
        

In [251]:
for s in range(1,len(bins)-1):
    for name in bins[s]:
        if name not in bins[s-1]:
            if name not in pre_brady.keys():
                pre_brady[name] = []
            for y in range(0,s):
                pre_brady[name] += [get_stats(name, dfs[y], years[y])]
                   
        if name not in bins[s+1]:
            if name not in post_brady.keys():
                post_brady[name] = []
            for y in range(s+1,len(bins)):
                post_brady[name] += [get_stats(name, dfs[y], years[y])]       
                
        if True:
            if name not in with_brady.keys():
                with_brady[name] = [get_stats(name, dfs[s], years[s])]
            else:
                with_brady[name] += [get_stats(name, dfs[s], years[s])]
#create the final bin              
for x in bin_2020:
    if x not in with_brady.keys():
        with_brady[x] = [get_stats(x, df20, 2020)]
    else:
        with_brady[x] += [get_stats(x, df20, 2020)]
        

In [252]:
#Get rid of brady in the pre bin to avoid confusion
pre_brady = {key:val for key, val in pre_brady.items() if key != 'T.BRADY'}

In [253]:
print('pre')
print(pre_brady.items())
print('Post')
print(post_brady.items())
print('with')
print(list(with_brady.items()))


pre
dict_items([('B.LAFELL', [('13', 620, 5)]), ('T.WRIGHT', [('13', 517, 5)]), ('B.TYMS', [('13', 12, 0)]), ('J.GRAY', [('13', 0, 0)]), ('K.MARTIN', [('13', 253, 2), ('14', 78, 0)]), ('D.LEWIS', [('13', 0, 0), ('14', 0, 0)]), ('J.WHITE', [('13', 0, 0), ('14', 23, 0)]), ('S.CHANDLER', [('13', 655, 2), ('14', 475, 3)]), ('S.JACKSON', [('13', 191, 1), ('14', 148, 0)]), ('M.WILLIAMS', [('13', 199, 1), ('14', 0, 0)]), ('T.CADET', [('13', 5, 1), ('14', 297, 1)]), ('C.HARPER', [('13', 0, 0), ('14', 0, 0)]), ('A.CLEVELAND', [('13', 0, 0), ('14', 0, 0)]), ('C.HOGAN', [('13', 83, 0), ('14', 409, 4), ('15', 448, 2)]), ('M.MITCHELL', [('13', 0, 0), ('14', 0, 0), ('15', 0, 0)]), ('M.BENNETT', [('13', 768, 6), ('14', 923, 5), ('15', 430, 3)]), ('J.DEVELIN', [('13', 62, 0), ('14', 43, 0), ('15', 0, 0)]), ('M.FLOYD', [('13', 1141, 4), ('14', 1670, 12), ('15', 1393, 9)]), ('M.LENGEL', [('13', 0, 0), ('14', 0, 0), ('15', 0, 0)]), ('R.BURKHEAD', [('13', 0, 0), ('14', 49, 0), ('15', 94, 1), ('16', 114, 0

In [254]:
players = pd.DataFrame(columns = ['Name', 'Bucket', 'Year', 'Yards', 'TD'])


for key, value in pre_brady.items():
    for x in range(len(value)):
        players = players.append({'Name':key, 'Bucket':'Pre', 'Year':value[x][0], \
                                 'Yards':value[x][1], 'TD':value[x][2]}, ignore_index = True)

for key, value in with_brady.items():
    for x in range(len(value)):
        players = players.append({'Name':key, 'Bucket':'With', 'Year':value[x][0], \
                                 'Yards':value[x][1], 'TD':value[x][2]}, ignore_index = True)

for key, value in post_brady.items():
    for x in range(len(value)):
        players = players.append({'Name':key, 'Bucket':'Post', 'Year':value[x][0], \
                                 'Yards':value[x][1], 'TD':value[x][2]}, ignore_index = True)
    

    



master = []
for x in bins:
    for s in x:
        if s not in master:
            master.append(s)


players = players.sort_values(by = ['Name', 'Year'])
print(players[players['Name'] == 'D.AMENDOLA'])


           Name Bucket Year Yards TD
163  D.AMENDOLA   With   13   633  2
164  D.AMENDOLA   With   14   193  1
165  D.AMENDOLA   With   15   653  3
166  D.AMENDOLA   With   16   238  4
167  D.AMENDOLA   With   17   637  2
423  D.AMENDOLA   Post   18   565  1
424  D.AMENDOLA   Post   19   662  1
425  D.AMENDOLA   Post   20   564  0
